In [1]:
import configparser
import os
import logging
import time

import hmac
import hashlib
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from requests import Response
from urllib.parse import urlencode
from typing import Optional, Dict, Any

In [2]:
class RESTClient:
    """Provide a Client for REST APIs."""
    
    def __init__(self, endpoint: str, path: str):
        self._endpoint = endpoint
        self._path = path

        
    def get_data(self, params: Optional[Dict[str, Any]] = None) -> Any:
        """Executes an HTTPS GET request."""
        
        try:
            response = requests.get(self._endpoint + self._path, params)
        except requests.exceptions.Timeout:
            logging.warning("Request has timed out")
        except requests.exceptions.ConnectionError:
            logging.warning("Request has faced a connection error")
            
        return self._process_response(response)

        
    def _process_response(self, response: Response) -> Any:
        """Processes the response the server sends to the clients request."""
        
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            return data

In [3]:
class BinanceFuturesClient(RESTClient):
    """Hold key information for Binance Spot Exchange."""

    def __init__(self):
        super().__init__('https://fapi.binance.com/', 'fapi/v1/klines')
    
    
class ByBitFuturesClient(RESTClient):
    """Hold key information for ByBit Futures Exchanges."""
    
    def __init__(self):
        super().__init__('https://api.bybit.com/', 'derivatives/v3/public/funding/history-funding-rate')

In [9]:
def fetch_historical_data(ec: RESTClient, params: dict) -> pd.DataFrame:

    funding = None
    response = ec.get_data(params)
    funding = pd.DataFrame(response['result']['list'], columns = \
                           ['fundingRateTimestamp', 'fundingRate'])
    funding['fundingRate'] = funding['fundingRate'].astype(float)

    return funding

In [12]:
ec = ByBitFuturesClient()

market_name = 'BTCUSDT'
delta = timedelta(hours=800)
start_time = datetime(2022, 9, 8, 20, 0, 0, 0)
end_time = start_time + delta

funding_bybit = pd.DataFrame()

while start_time < datetime.now():
    
    params = {
        'symbol': market_name,
        'start': int((start_time-timedelta(hours=4)).timestamp()*1000),
        'end': int(end_time.timestamp()*1000)
    }

    tmp_funding = fetch_historical_data(ec, params)

    funding_bybit = pd.concat([funding_bybit, tmp_funding], ignore_index=True)

    start_time = end_time
    end_time = end_time + delta

print(funding_bybit)

     fundingRateTimestamp  fundingRate
0           1684454400000     0.000071
1           1684425600000     0.000100
2           1684396800000    -0.000010
3           1684368000000     0.000058
4           1684339200000     0.000077
...                   ...          ...
1595        1678838400000     0.000100
1596        1678809600000     0.000100
1597        1678780800000     0.000100
1598        1678752000000     0.000122
1599        1678723200000     0.000100

[1600 rows x 2 columns]
